In [50]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

# Входной файл после препроцессинга (без скейлинга)
DATA_PATH = "data/preprocessed_without_scaling.csv"

# Выходные файлы с фичами (после скейлинга)
WORKOUT_OUT = "data/workout.csv"
MEAL_OUT    = "data/meal.csv"



In [51]:
df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()


(19823, 36)


,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,Daily meals frequency,Calories,sugar_g,serving_size_g,rating,Name of Exercise,Sets,Reps,Benefit,Burns Calories (per 30 min),Target Muscle Group,Equipment Needed,Difficulty Level,Body Part,Type of Muscle,Workout,sodium_g,cholesterol_g,Carbs,Proteins,Fats,meal_name
0,35,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,1.50,4,2,24.87,3,1806.0,31.77,120.47,1.31,Decline Push-ups,5,21,Improves shoulder health and posture,342.58,"Shoulders, Triceps",Cable Machine,3,Legs,Lats,Dumbbell flyes,1.72994,0.28505,267.68,106.05,71.63,Grilled Vegan Lunch
1,23,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,1.90,4,2,23.48,3,1577.0,12.34,109.15,1.92,Bear Crawls,4,16,Strengthens lower abs,357.16,"Back, Core, Shoulders",Step or Box,2,Chest,Lats,Lateral raises,0.69308,0.30061,214.32,85.41,56.97,Fried Vegetarian Lunch
2,33,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,1.88,3,1,21.15,2,1608.0,42.81,399.43,4.70,Dips,5,22,Builds chest strength,359.63,"Quadriceps, Glutes",Step or Box,2,Arms,Grip Strength,Standing calf raises,2.14248,0.21542,246.04,98.11,65.48,Boiled Paleo Breakfast
3,39,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,2.50,4,2,32.45,3,2657.0,9.34,314.31,4.85,Mountain Climbers,4,17,Improves coordination and cardiovascular health,351.65,"Biceps, Forearms",Parallel Bars or Chair,3,Shoulders,Upper,Incline dumbbell flyes,0.12320,0.00970,203.22,80.84,54.56,Fried Paleo Lunch
4,45,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,2.91,4,2,14.83,3,1470.0,23.78,99.22,3.07,Bicep Curls,5,15,Targets obliques and improves core rotation,329.36,"Chest, Triceps",Wall,3,Abs,Wrist Flexors,Military press,1.93511,0.11689,332.79,133.05,88.43,Baked Vegan Breakfast


In [52]:
df.columns

Index(['Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Max_BPM', 'Avg_BPM',
       'Resting_BPM', 'Session_Duration (hours)', 'Calories_Burned',
       'Workout_Type', 'Water_Intake (liters)',
       'Workout_Frequency (days/week)', 'Experience_Level', 'BMI',
       'Daily meals frequency', 'Calories', 'sugar_g', 'serving_size_g',
       'rating', 'Name of Exercise', 'Sets', 'Reps', 'Benefit',
       'Burns Calories (per 30 min)', 'Target Muscle Group',
       'Equipment Needed', 'Difficulty Level', 'Body Part', 'Type of Muscle',
       'Workout', 'sodium_g', 'cholesterol_g', 'Carbs', 'Proteins', 'Fats',
       'meal_name'],
      dtype='object')

BMR, PAL, TDEE, CaloriesPerDay, BMI

In [53]:
def add_user_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Добавляем базовые 'user-level' фичи:
    BMR, PAL, TDEE, BMI (если вдруг не было)
    """
    out = df.copy()

    weight = out["Weight (kg)"]
    height_cm = out["Height (m)"] * 100
    age = out["Age"]

    # пол: считаем, что 'male'/'female'
    is_male = out["Gender"].str.lower().str.startswith("m")

    bmr_male   = 10 * weight + 6.25 * height_cm - 5 * age + 5
    bmr_female = 10 * weight + 6.25 * height_cm - 5 * age - 161

    out["BMR"] = np.where(is_male, bmr_male, bmr_female)

    # PAL = 1.2 + 0.175 * Workout_Frequency (days/week)
    out["PAL"] = 1.2 + 0.175 * out["Workout_Frequency (days/week)"]

    # TDEE = BMR * PAL
    out["TDEE"] = out["BMR"] * out["PAL"]

    # Если вдруг BMI не был посчитан
    out["BMI"] = out["Weight (kg)"] / (out["Height (m)"] ** 2)

    return out


df = add_user_features(df)
df.head()



,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,Daily meals frequency,Calories,sugar_g,serving_size_g,rating,Name of Exercise,Sets,Reps,Benefit,Burns Calories (per 30 min),Target Muscle Group,Equipment Needed,Difficulty Level,Body Part,Type of Muscle,Workout,sodium_g,cholesterol_g,Carbs,Proteins,Fats,meal_name,BMR,PAL,TDEE
0,35,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,1.50,4,2,24.870447,3,1806.0,31.77,120.47,1.31,Decline Push-ups,5,21,Improves shoulder health and posture,342.58,"Shoulders, Triceps",Cable Machine,3,Legs,Lats,Dumbbell flyes,1.72994,0.28505,267.68,106.05,71.63,Grilled Vegan Lunch,1495.20,1.900,2840.88000
1,23,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,1.90,4,2,23.479709,3,1577.0,12.34,109.15,1.92,Bear Crawls,4,16,Strengthens lower abs,357.16,"Back, Core, Shoulders",Step or Box,2,Chest,Lats,Lateral raises,0.69308,0.30061,214.32,85.41,56.97,Fried Vegetarian Lunch,1256.85,1.900,2388.01500
2,33,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,1.88,3,1,21.148123,2,1608.0,42.81,399.43,4.70,Dips,5,22,Builds chest strength,359.63,"Quadriceps, Glutes",Step or Box,2,Arms,Grip Strength,Standing calf raises,2.14248,0.21542,246.04,98.11,65.48,Boiled Paleo Breakfast,1307.55,1.725,2255.52375
3,39,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,2.50,4,2,32.449827,3,2657.0,9.34,314.31,4.85,Mountain Climbers,4,17,Improves coordination and cardiovascular health,351.65,"Biceps, Forearms",Parallel Bars or Chair,3,Shoulders,Upper,Incline dumbbell flyes,0.12320,0.00970,203.22,80.84,54.56,Fried Paleo Lunch,1644.30,1.900,3124.17000
4,45,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,2.91,4,2,14.831372,3,1470.0,23.78,99.22,3.07,Bicep Curls,5,15,Targets obliques and improves core rotation,329.36,"Chest, Triceps",Wall,3,Abs,Wrist Flexors,Military press,1.93511,0.11689,332.79,133.05,88.43,Baked Vegan Breakfast,1479.20,1.900,2810.48000


In [54]:
new_user = pd.DataFrame([{
    'Age': 38,
    'Gender': 'Female',
    'Weight (kg)': 100,
    'Height (m)': 1.70,
    'BMI': 34.6,
    'Workout_Frequency (days/week)': 3,
    'Daily meals frequency': 2,
    'diet_type': 'Low-Carb',
    'Experience_Level': 2,
    'Goal': 'Loss',
    'WeightChange (kg)': 5,
    'GoalDays': 30
}])


new_user = add_user_features(new_user)

new_user['CalorieChange'] = new_user['WeightChange (kg)'] * 7700
new_user['CloriesToBurnTraining'] = new_user['CalorieChange'] * 0.5
new_user['CloriesReducedFromFood'] = new_user['CalorieChange'] * 0.5
new_user['CaloriesPerDay'] = new_user['TDEE'] - (new_user['CloriesReducedFromFood'] / new_user['GoalDays'])
new_user['TotalWorkouts'] = round(new_user['Workout_Frequency (days/week)'] * new_user['GoalDays'] / 7)
new_user['CaloriesPerWorkout'] = new_user['CloriesToBurnTraining'] / new_user['TotalWorkouts']
new_user["Meal_target"] = new_user["CaloriesPerDay"] / new_user["Daily meals frequency"]

new_user.head()

,Age,Gender,Weight (kg),Height (m),BMI,Workout_Frequency (days/week),Daily meals frequency,diet_type,Experience_Level,Goal,WeightChange (kg),GoalDays,BMR,PAL,TDEE,CalorieChange,CloriesToBurnTraining,CloriesReducedFromFood,CaloriesPerDay,TotalWorkouts,CaloriesPerWorkout,Meal_target
0,38,Female,100,1.7,34.602076,3,2,Low-Carb,2,Loss,5,30,1711.5,1.725,2952.3375,38500,19250.0,19250.0,2310.670833,13.0,1480.769231,1155.335417


Workout feature engineering (E, I, S, D, R)

In [55]:
def add_workout_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Строим фичи для тренировок:
    E - Energy Consumption
    I - Intensity
    S - Power component
    D - Duration (нормированная)
    R - Risk (1 - penalties)
    """
    out = df.copy()

    # --- E: Energy consumption ---
    # E_raw
    cals = out["Calories_Burned"]
    out["E_raw"] = (cals - cals.min()) / (cals.max() - cals.min())

    # E_eff
    burn30 = out["Burns Calories (per 30 min)"]
    out["E_eff"] = (burn30 - burn30.min()) / (burn30.max() - burn30.min())

    out["E"] = 0.5 * out["E_raw"] + 0.5 * out["E_eff"]

    # --- I: Intensity (HRR) ---
    out["pct_HRR"] = (out["Avg_BPM"] - out["Resting_BPM"]) / (out["Max_BPM"] - out["Resting_BPM"])
    hrr = out["pct_HRR"]
    out["I"] = (hrr - hrr.min()) / (hrr.max() - hrr.min())

    # --- S: Power component ---
    out["workload"] = out["Sets"] * out["Reps"]
    S = out["workload"] / out["Difficulty Level"]
    out["S"] = (S - S.min()) / (S.max() - S.min())

    # --- D: Duration (нормируем длину тренировки) ---
    out["Duration_min"] = out["Session_Duration (hours)"] * 60
    dur = out["Duration_min"]
    out["D"] = (dur - dur.min()) / (dur.max() - dur.min())

    # --- R: Risk = 1 - (0.4*pen_age + 0.3*pen_bmi + 0.2*pen_hrr + 0.1*pen_skill) ---

    # 1) возраст
    age = out["Age"]
    out["pen_age"] = (age - age.min()) / (age.max() - age.min())

    # 2) BMI
    bmi = out["BMI"]
    out["pen_bmi"] = (bmi - bmi.min()) / (bmi.max() - bmi.min())

    # 3) HRR (верхняя часть распределения)
    hrr90 = hrr.quantile(0.9)
    hrr_max = hrr.max()
    out["pen_hrr"] = np.where(
        hrr > hrr90,
        (hrr - hrr90) / (hrr_max - hrr90),
        0.0
    )

    # 4) Skill penalty (сложность > опыт)
    out["pen_skill"] = np.maximum(0, out["Difficulty Level"] - out["Experience_Level"])

    out["R"] = 1 - (
        0.4 * out["pen_age"]
      + 0.3 * out["pen_bmi"]
      + 0.2 * out["pen_hrr"]
      + 0.1 * out["pen_skill"]
    )

    return out


df = add_workout_features(df)
df[["E", "I", "S", "D", "R"]].head()


,E,I,S,D,R
0,0.570217,0.526466,0.250000,0.333333,0.637294
1,0.740993,0.256055,0.225000,0.575163,0.862504
2,0.549660,0.289199,0.416667,0.274510,0.685581
3,0.660172,0.530620,0.147222,0.398693,0.539662
4,0.560736,0.422805,0.166667,0.385621,0.620914


Meal feature engineering (C, P, MacroMatch, ED, F)

In [56]:
def add_meal_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Строим фичи для еды:
    C  - Calorie Fit (насколько блюдо попадает в целевые калории)
    P  - Proteins per meal
    MacroMatch - насколько БЖУ близки к целевым долям
    ED - Energy Density (kcal/gram)
    F  - Food Safety (по сахару, натрию, холестерину)
    """
    out = df.copy()

    # --- C: Calorie Fit ---
    # Целевая калорийность одного приёма пищи
    meal_target = new_user["Meal_target"].iloc[0]
    out["C"] = 1 - (out["Calories"]/out["Daily meals frequency"] - meal_target).abs() / meal_target


    # --- P: Proteins per meal ---
    P = out["Proteins"] / out["Daily meals frequency"]
    out["P"] = (P - P.min()) / (P.max() - P.min())

    # --- M: MacroMatch ---
    # пересчитываем БЖУ в калории
    out["cal_from_protein"] = out["Proteins"] * 4
    out["cal_from_carbs"] = out["Carbs"] * 4
    out["cal_from_fats"] = out["Fats"] * 9

    total_macro_cal = (
        out["cal_from_protein"] +
        out["cal_from_carbs"] +
        out["cal_from_fats"]
    ).replace(0, np.nan)

    out["pct_p"] = out["cal_from_protein"] / total_macro_cal
    out["pct_c"] = out["cal_from_carbs"] / total_macro_cal
    out["pct_f"] = out["cal_from_fats"] / total_macro_cal

    # Целевые доли
    if new_user['Goal'].iloc[0] == 'Loss':
        target_p, target_c, target_f = 0.3, 0.35, 0.35
    elif new_user['Goal'].iloc[0] == 'Maintain':
        target_p, target_c, target_f = 0.2, 0.5, 0.3
    elif new_user['Goal'].iloc[0] == 'Gain':
        target_p, target_c, target_f = 0.25, 0.55, 0.2

    out["M"] = 1 - (1 / 3) * (
        ((out["pct_p"] - target_p).abs())/target_p +
        ((out["pct_c"] - target_c).abs())/target_c +
        ((out["pct_f"] - target_f).abs())/target_f
    )

    # --- ED: Energy Density ---
    ED = out["Calories"] / (out["serving_size_g"] * out['Daily meals frequency'])
    out["ED"] = (ED - ED.min()) / (ED.max() - ED.min())

    # --- F: Food Safety ---
    sugar_90 = out["sugar_g"].quantile(0.9)
    sodium_90 = out["sodium_g"].quantile(0.9)
    chol_90   = out["cholesterol_g"].quantile(0.9)

    out["F"] = 1 - (1/3) * (
        (out["sugar_g"] / sugar_90).clip(upper=1) +
        (out["sodium_g"] / sodium_90).clip(upper=1) +
        (out["cholesterol_g"] / chol_90).clip(upper=1)
    )

    return out


df = add_meal_features(df)
df[["C", "P", "M", "ED", "F"]].head()


,C,P,M,ED,F
0,0.521061,0.342572,0.697313,0.258372,0.173291
1,0.454991,0.211067,0.696730,0.247696,0.471934
2,0.695902,0.604530,0.697190,0.082439,0.096575
3,0.766588,0.181950,0.699098,0.129894,0.899428
4,0.424119,0.514598,0.697327,0.254918,0.392439


We divide it into workout features and meal features.

In [57]:
# Минимальный набор для workout-модели:
workout_features = df[[
    "Name of Exercise",
    "Workout_Type",
    "Target Muscle Group",
    "Body Part",
    "Type of Muscle",
    "Difficulty Level",
    "Calories_Burned",
    "Sets", "Reps",
    "Burns Calories (per 30 min)",
    "E", "I", "S", "D", "R"
]].copy()

# Минимальный набор для meal-модели:
meal_features = df[[
    "meal_name",
    "Calories",
    "Proteins",
    "Carbs",
    "Fats",
    "serving_size_g",
    "sodium_g",
    "cholesterol_g",
    "C", "P", "M", "ED", "F"
]].copy()


In [58]:
workout_features.to_csv(WORKOUT_OUT, index=False)
print("Saved workout features to:", WORKOUT_OUT)
meal_features.to_csv(MEAL_OUT, index=False)
print("Saved meal features to:", MEAL_OUT)

new_user.to_csv('data/new_user.csv', index=False)

df.to_csv('data/dataset_with_features.csv', index=False)

Saved workout features to: data/workout.csv
Saved meal features to: data/meal.csv


Scaling (StandardScaler)-------------------------------------------------------------------------------------------------------------------------------------
перенести в модельку

Workout Scaling

In [59]:
workout_num_cols = ["E", "I", "S", "D", "R", "Calories_Burned",
    "Sets", "Reps",
    "Burns Calories (per 30 min)"]

scaler_workout = StandardScaler()
workout_scaled = scaler_workout.fit_transform(workout_features[workout_num_cols])

workout_scaled_df = pd.DataFrame(
    workout_scaled,
    columns=[col + "_scaled" for col in workout_num_cols],
    index=workout_features.index
)

Meal Scaling

In [60]:
meal_num_cols = ["Calories", "Proteins", "Carbs", "Fats",
                 "serving_size_g", "sodium_g", "cholesterol_g",
                 "C", "P", "M", "ED", "F"]

scaler_meal = StandardScaler()
meal_scaled = scaler_meal.fit_transform(meal_features[meal_num_cols])

meal_scaled_df = pd.DataFrame(
    meal_scaled,
    columns=[col + "_scaled" for col in meal_num_cols],
    index=meal_features.index
)
